# Introduction to Floating Point Issues in Computation
## This notebook is a walkthrough and tutorial of floating point related topics, largely inspired (and closely following) the paper described below:

Link: http://www.validlab.com/goldberg/paper.ps Summary from the paper "Note – This document is an edited reprint of the paper What Every Computer Scientist Should Know About Floating-Point Arithmetic, by David Goldberg, published in the March, 1991 issue of Computing Surveys. Copyright 1991, Association for Computing Machinery, Inc., reprinted by permission."

More details available from:
http://grouper.ieee.org/groups/754/



## The notebook organization will largely mirror the scheme from the above paper
- Motivation
- Part I: Rounding Errors
- Part II: IEEE 754 Standard
- Part III: Details (and other issues)

# Motivation
As implied in the title of the key reference "What Every Computer Scientist Should Know About Floating-Point Arithmetic" having a working understanding about this topic seems to be fundamental to essentially anyone working with computation.  Even though the details might be somewhat hidden inside the hardware and software implementations, the issues should be understood since they can manifest in various ways in practice.  This notebook attempts to extract key ideas from the reference(s) and provide some exercises to gain some familiarity with the points raised.

# Part I: Rounding Errors
Rounding Error:
Basic problem.  Infinite number of real numbers, finite number of bits.

Real number → Representable by a floating point representation (but sometimes not exactly)
When exact, it is called a floating point number.

## Basic representation 

$ \pm d_0.d_1 d_2 d_3 d_4 d_{p-1} \times \beta^e$ represents the number
$ \pm ( d_0+d_1 \beta^{-1} + \ldots + d_{p-1} \beta^{-(p-1)} ) \beta^e $

### Exercise
Show that the number of bits required to encode the above number is
$ \lceil log_{2} (e_{max}-e_{min}+1) \rceil + \lceil log_{2}\beta^{p} \rceil + 1 $  
where $\lceil x \rceil$ is the ceiling function applied to $x$, $e_{min}$ is the minimum exponent, $e_{max}$ is the maximum exponent, $\beta$ is the base (like 2 or 10), and $p$ is the number of places we are using to represent numbers in our system.

### Response:

**This section is an intentional small break in the text.  You can use it to type in your response to the exercise if you want.  I placed this buffer before each solution that I worked out, so you can hide my solution and work on your own first before comparing.  The solution section appears immediately below.**





#### End response

### Solution

We require that $2^{N} > R$, where $N$ is the number of bit we require and $R$ is the real number we are trying to represent using binary.  
Taking the base 2 logarithm of both sides yields:
$N > log_2 R$.  If we take the ceiling of the right hand side 
(the smallest integer such that the inequality is satisfied)
we obtain the desired result shown in the theory.  The additional 1 comes from the bit needed for representing the sign.

In [13]:
# Exploration code:
# Example of bits required
import numpy as np
emax =  5  
emin = -4
beta = 10 # base10
p = 4     # four figures

#TODO:  Implement the theory into the missing python code
num_bits_exponent = np.ceil(np.log2(emax-emin+1))
num_bits_significand = np.ceil(np.log2(beta**p))
num_bits_sign = 1
total_bits = num_bits_exponent + num_bits_significand + num_bits_sign

print('num_bits_exponent = ', num_bits_exponent)
print('num_bits_significand = ', num_bits_significand)
print('num_bits_sign = ', num_bits_sign)
print('total_bits = ', total_bits)

num_bits_exponent =  4.0
num_bits_significand =  14.0
num_bits_sign =  1
total_bits =  19.0


## Normalized Representation
When the number starts with a 1, this is called "normalized."  That is to say that the $d_{0}$ in the above discussion of representation is set to 1.  It should be mentioned that in certain representations, this "1" is implied, but for now, let us assume it is required to be present (i.e. one bit in a memory location is used for this).

### Exercise
Show that if $\beta=2$, p=3, $e_{min}=-1$ and $e_{max}=2$, there are 16 normalized numbers that can be represented

### Response:






#### End response

### Solution
For p = 3:  
100  
101  
110  
111  
There is a group of these for each exponent:  $\beta^{-1}$, $\beta^{0}$, $\beta^{1}$, and $\beta^{2}$.
Therefore we have 4x4 = 16 possible normalized numbers.

### Exercise

Given  that the representation:
$ \pm d_0.d_1 d_2 d_3 d_4 d_{p-1} \times \beta^e$ represents the number
$ \pm ( d_0+d_1 \beta^{-1} + \ldots + d_{p-1} \beta^{-(p-1)} ) \beta^e $

Write down what each of the 16 normalized encoding represent in decimal numbers (assume all positive).  Recall that for decimal (base 10), we would have a representation like this for the number 120

$1.20 \times 10^2 = 120 = 1 \times 10^0(10^2) + 2 \times 10^{-1}(10^2) + 0 \times 10^{-2}(10^2)$

### Response:






#### End response

### Solution
First two are done explicitly, remaining are computed using the Python code below.  As a side note, observe that the smallest number that can be represented in this case is 0.5.  During program execution, an exception called "Underflow" is triggered when a computation yields a result that will fall into the range of numbers less than 0.5.

(exponent = -1.  Encoded as 00)  
100 : 00  = $1 \times 2^{0} 2^{-1} + 0 \times 2^{-1} 2^{-1} + 0 \times 2^{-2} 2^{-1} = 0.5$  
101 : 00  = $1 \times 2^{0} 2^{-1} + 0 \times 2^{-1} 2^{-1} + 1 \times 2^{-2} 2^{-1} = 0.5 + 0.125 = 0.625$  
110 : 00  
111 : 00  

(exponent = 0.  Encoded as 01)  
100 : 01   =   
101 : 01  
110 : 01  
111 : 01  

(exponent = 0.  Encoded as 10)  
100 : 10  
101 : 10  
110 : 10  
111 : 10  

(exponent = 1.  Encoded as 11)  
100 : 11  
101 : 11  
110 : 11  
111 : 11

In [14]:
# Exercise for normalized numbers
import numpy as np

# We want to generate the decimal equivalent of the above encodings:
def normalized2decimal(prefix, exponent, base):
    
    res = 0
    pos = 0
    for iSym in prefix:
        res = res+ 1.0*int(iSym)*(base**pos)*base**(exponent)
        pos = pos-1
        
    return res

# Try out some cases:

print(normalized2decimal('100',-1,2))
print(normalized2decimal('101',-1,2))
print(normalized2decimal('110',-1,2))
print(normalized2decimal('111',-1,2))
print()
print(normalized2decimal('100',0,2))
print(normalized2decimal('101',0,2))
print(normalized2decimal('110',0,2))
print(normalized2decimal('111',0,2))
print()
print(normalized2decimal('100',1,2))
print(normalized2decimal('101',1,2))
print(normalized2decimal('110',1,2))
print(normalized2decimal('111',1,2))
print()
print(normalized2decimal('100',2,2))
print(normalized2decimal('101',2,2))
print(normalized2decimal('110',2,2))
print(normalized2decimal('111',2,2))
   

0.5
0.625
0.75
0.875

1.0
1.25
1.5
1.75

2.0
2.5
3.0
3.5

4.0
5.0
6.0
7.0


In the normalized representation, the number zero has a special challenge because of the requirement to have a leading one.  Therefore, a special encoding is used.  One of the exponents is reserved to represent (or indicate) the number zero.  Therefore, continuing the earlier representation scheme, here is the complete list of 16 normalized numbers with zero included.  The exponent bits are shown to the right of the colon symbol below.  Notice that the exponent range is reduced by one to accomodate the encoding for zero. 

(special zero exponent = -2.  Encoded as 00)    
100 : 00  = Zero    
101 : 00  = Not valid  
110 : 00  = Not valid  
111 : 00  = Not valid  

(exponent = -1. Encoded as 01)  
100 : 01  = $1 \times 2^{0} 2^{-1} + 0 \times 2^{-1} 2^{-1} + 0 \times 2^{-2} 2^{-1} = 0.5$    
101 : 01  = $1 \times 2^{0} 2^{-1} + 0 \times 2^{-1} 2^{-1} + 1 \times 2^{-2} 2^{-1} = 0.5 + 0.125 = 0.625$     
110 : 01  
111 : 01  

(exponent = 0. Encoded as 10)  
100 : 10 =  
101 : 10  
110 : 10  
111 : 10  

(exponent = 1. Encoded as 11)  
100 : 11 =  
101 : 11  
110 : 11  
111 : 11  

## Relative Error, Units in the Last Place (ULPs), Machine Epsilon

At this point, the general theme should be emerging in that an arbitrary real number ("infinite precision") is approximated in the computed by a floating point representation.  There are gaps between each floating point representation when compared to the real-numbers, such that a given real number may fall between two floating point representations and thus, will need to either be assigned to the lower one or the upper one.  Thus, the error is bounded to be no more than $\frac{1}{2}$ of the spacing at that particular interval, which from the above analysis is clearly not uniform, but depends on the exponent that applies in that particular range.

There is an ultimate limit to this, which is known as *machine epsilon*, $\epsilon$, which is the smallest interval that can be represented by the machine implementation.

The *units in the last place* (ulps) is absolute error between the real number in question and the closest floating point number.  For example, consider 0.03412 as the real number, and 0.034 as the floating point representation (assuming for a moment this number can be represented as shown).  Then we have $0.03412-0.034=0.12ulps$.

*Relative error* is this difference, divided by the real number itself.  So in this case, we would have $\frac{0.00012}{0.03412}$.

## Cancellation Effects

One of the main areas that need to be understood is the limitations of floating-point representations for certain types of computations, and strategies to remedy the impact.

Basically, without appropriate measures, subtractions between two close numbers can produce large relative errors.  We'll go through several examples, and then explain some remedies.  These remedies may be "built-in" to the processing logic or may required interventional by the developer.  However, it is well-worth understanding the issues and potential traps to avoid.

The main culprit is the following.  In any sequence of calculations, you may introduce round off errors.  If you end up with a calculation that subtracts two terms, each of which have round-off errors, then the result will be significantly different than the "infinite precision" version of the calculation.  The main point to keep in mind is that this only an issue when you have round off errors (i.e. your intermediate result was mapped to the nearest number that could be represented in floating point.).

To illustrate the second type of cancellation first, let us assume we have two exactly representable numbers using base-2 for convenience and use the above normalized sequence as our represent out computations.  The difference between 1.5 and 1.0 = 0.5.  This is an exactly representable number, so there is no issue.  However, take the case of the difference between 1.6 and 1.0 = 0.6 in exact terms.  Now, 1.6 will be rounded to 1.5, and the difference taken will be 1.5-1.0 = 0.5.  This is an error of $\frac{0.6-0.5}{0.6}\approx16.7\%$.  

This is bad, but the absolute worst possible result is if the two terms round to the same floating point number, say we were computing 1.5-1.4.  The real answer is 0.1, but the result after 1.4 is rounded to 1.5 is *ZERO*.  Clearly, the use of this intermediate result in a downstream calculation would be highly suspect if not "catastrophic" as described in the Goldberg reference mentioned in the beginning.

## Exact vs. Approximate Calculations

Up to this point, we haven't emphasized the subtleties of exact vs. approximate calculations.  Indeed, this is a complete subject in its own right called *Numerical Analysis*.  However, some key distinctions need to be made.  We should always keep in mind that there is some sort of exact calculation we are attempting to reproduce using the computer program/algorithm.  In fact, there is a further level to this to consider, namely how one actually computes a function in the first place even if you have infinite precision available?  For example, let $f(x)=\sqrt{x}$ be the function we wish to compute, for $x\ge0$.  Here is one possible approach we could take:

1) Intialize m = 0  
2) Compute res_estimate_low_2 = m\*m  
3) Compute res_estimate_high_2 = (m+1)\*(m+1)  
4) If res_estimate_low_2 = x then return m    
5) If res_estimate_low_2 < x and res_estimate_high_2 > x then return m+1/2  
6) Else m=m+1, repeat from step (2).  

The above algorithm can be done by hand and on machines with finite precision without error assuming representing the intermediate calculations do not exceed the memory for representing a given number. 

## Guard Digits

We now begin some discussion about how these issues are addressed in practice.


## Exactly Rounded Operations

This section discusses other approaches and consequences to floating-point issues.

# Part II: IEEE 754

Discussion of the floating point standard.

# Part III: Details (and other issues)

Proof details, and other issues.